In [1]:
import numpy as np
import pickle
import tensorflow as tf
import random
from Vel2Img import *
import os

In [2]:
#load the data set
with open('./data/velocity.pickle', 'rb') as handle:
    velocities = pickle.load(handle)

for i in range(velocities.shape[0]):
    velocities[i,:,:,:]=(velocities[i,:,:,:]-velocities[i,:,:,:].min())/(velocities[i,:,:,:].max()-velocities[i,:,:,:].min())

In [3]:
velocities[1,:,:,:].max()

1.0

In [4]:
#Build the discriminator

def convolution2d(input, biases, weights, strides, padding_kind='SAME'):
    input = tf.nn.conv2d(input, weights, [1,strides,strides,1], padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    input = tf.nn.leaky_relu(input)
    return input

def discriminator(x_image):
    
    #layer1: Convolution
    weights1=tf.Variable(tf.random_normal([12,12,2,2], stddev=0.01),name='d_Wconv1')
    #[filter_height, filter_width, in_channels, out_channels]
    #bias=out_channels
    bias1=tf.Variable(tf.random_normal([2],stddev=0.01), name='d_Bconv1')
    stride1=2
    out1=convolution2d(x_image,bias1,weights1,stride1)
    print(out1.shape)
    #layer2: Convolution
    weights2=tf.Variable(tf.random_normal([6,6,2,4], stddev=0.01),name='d_Wconv2')
    bias2=tf.Variable(tf.random_normal([4], stddev=0.01),name='d_Bconv2')
    stride2=4
    out2=convolution2d(out1,bias2,weights2,stride2)
    print(out2.shape)
    #layer3: Convolution
    weights3=tf.Variable(tf.random_normal([4,4,4,8],stddev=0.01), name='d_Wconv3')
    bias3=tf.Variable(tf.random_normal([8],stddev=0.01), name='d_Bconv3')
    stride3=2
    out3=convolution2d(out2,bias3,weights3,stride3)
    print(out3.shape)
    #layer4: Convolution
    weights4=tf.Variable(tf.random_normal([3,3,8,16], stddev=0.01),name='d_Wconv4')#weights==filters
    bias4=tf.Variable(tf.random_normal([16], stddev=0.01),name='d_Bconv4')
    stride4=2
    out4=convolution2d(out3,bias4,weights4,stride4)
    print(out4.shape)
    #layer5: Fully Connected Layer
    out4 = tf.reshape(out4, shape=[-1, 64 ]) # flatten
    fc_1weights = tf.Variable(tf.random_normal([64, 1],stddev=0.01), name='d_WFCN1')
    fc_1bias   = tf.Variable(tf.random_normal([1], stddev=0.01),name='d_BFCN1')
    fc1 = tf.add(tf.matmul(out4, fc_1weights), fc_1bias)
    fc1 = tf.nn.sigmoid(fc1)
    
    return fc1

In [5]:
#Build the Generator

def deconvolution2d(input, weights, biases,outputShape, strides, padding_kind='SAME',activation='sigmoid'):
    # needed for dynamic shape with deconvolution
    dynamicBatchSize = tf.shape(input)[0]
    deconvShape = tf.stack([dynamicBatchSize, outputShape[1], outputShape[2], outputShape[3]])
    input = tf.nn.conv2d_transpose(input, weights, deconvShape, [1,strides,strides,1], padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    if activation =='leaky':
        input = tf.nn.leaky_relu(input)
    elif activation=='tanh':
        input = tf.nn.tanh(input)
    elif activation=='sigmoid':
        input = tf.nn.sigmoid(input)
    return input

def generator(noise):
    #layer1: DeConvolution
    weights5=tf.Variable(tf.random_normal([3,3,8,16],stddev=0.1), name='g_Wdeconv1')#weights==filters
    #[filter_height, filter_width, in_channels, out_channels]
    #bias=out_channels
    bias5=tf.Variable(tf.random_normal([8], stddev=0.1),name='g_Bdeconv1')
    stride5=2
    deconv1=deconvolution2d(noise,weights5,bias5,[None, 4,4, 8],stride5)

    #layer2: DeConvolution
    weights6=tf.Variable(tf.random_normal([4,4,4,8], stddev=0.1),name='g_Wdeconv2')#weights==filters
    bias6=tf.Variable(tf.random_normal([4],stddev=0.1), name='g_Bdeconv2')
    stride6=2
    deconv2=deconvolution2d(deconv1,weights6,bias6,[None, 8,8, 4],stride6)

    #layer3: DeConvolution
    weights7=tf.Variable(tf.random_normal([6,6,2,4], stddev=0.1), name='g_Wdeconv3')#weights==filters
    bias7=tf.Variable(tf.random_normal([2], stddev=0.1), name='g_Bdeconv3')
    stride7=4
    deconv3=deconvolution2d(deconv2,weights7,bias7,[None, 32,32, 2],stride7)

    #layer4: DeConvolution
    weights8=tf.Variable(tf.random_normal([12,12,2,2], stddev=0.1),name='g_Wdeconv4')#weights==filters
    bias8=tf.Variable(tf.random_normal([2], stddev=0.1), name='g_Bdeconv4')
    stride8=2
    xOut=deconvolution2d(deconv3,weights8,bias8,[None, 64,64, 2],stride8,activation='sigmoid')# based on GANHacks
    
    return xOut



In [6]:
#Training
batch_size = 10
sess = tf.Session()
ImageInput = tf.placeholder(tf.float32,shape = [None, 64,64, 2])
NoiseInput = tf.placeholder(tf.float32,shape=[None, 2,2, 16])


In [7]:
GeneratedImage = generator(NoiseInput) #GeneratedImage holds the generated images

RealImages = discriminator(ImageInput) #holds discriminator outputs (unnormalized) for the real images
FakeImages = discriminator(GeneratedImage) #will hold the discriminator output (unnormalized) for generated images

(?, 32, 32, 2)
(?, 8, 8, 4)
(?, 4, 4, 8)
(?, 2, 2, 16)
(?, 32, 32, 2)
(?, 8, 8, 4)
(?, 4, 4, 8)
(?, 2, 2, 16)


In [8]:
# Build Loss
Generator_loss = -tf.reduce_mean(tf.log(FakeImages))

Discriminator_loss= -tf.reduce_mean(tf.log(RealImages) + tf.log(1. - FakeImages))

In [9]:
# separate each network variables
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'd_' in var.name]
g_vars = [var for var in tvars if 'g_' in var.name]

In [10]:
learning_rate=1e-4
# Create the Optimiser
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
# Create training operations on the repective netork variable only
train_gen = optimizer_gen.minimize(Generator_loss, var_list=g_vars)
train_disc = optimizer_disc.minimize(Discriminator_loss, var_list=d_vars)

In [11]:
trainingEpochs=100
batchSize= 100
loadNum = len(velocities)
#creating Seesion starting training
print("Starting training...")
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# lets train for all epochs
for epoch in range(1,trainingEpochs+1):
    batch = []
    for currNo in range(0, batchSize):
        r = random.randint(0, loadNum-1)
        batch.append( velocities[r] )
        
    # Generate noise to feed to the generator
    z = np.random.uniform(-1., 1., size=[batchSize, 2,2,16])
        
    # Train
    fed_dict = {ImageInput: batch, NoiseInput: z}
    t,_, _, gl, dl = sess.run([GeneratedImage,train_gen, train_disc, Generator_loss, Discriminator_loss],
                                feed_dict=fed_dict)

    if epoch % 1 == 0 or epoch == 1:
        print('Epoch %i: Generator Loss: %f, Discriminator Loss: %f' % (epoch, gl, dl))

        outDir = "./Progress/"
        if not os.path.exists(outDir): 
            os.makedirs(outDir)   
        r = random.randint(0, batchSize-1)
        scipy.misc.toimage(np.reshape(z[r,:,:,:],(8,8)) , cmin=0.0, cmax=1.0).save("%s/noise_%d.png" % (outDir,epoch))
       # g = sess.run([GeneratedImage], feed_dict={NoiseInput:z[r:r+1,:,:,:] })
        g = np.reshape(t[r], newshape=( 64, 64, 2))
        generatedImage=velocityFieldToPng(g)

        scipy.misc.toimage( np.reshape(generatedImage, [64, 64, 3]) , cmin=0.0, cmax=1.0).save(outDir+'/genImg_'+str(epoch)+'.png')

print("Done...")


Starting training...
Epoch 1: Generator Loss: 0.691128, Discriminator Loss: 1.382547


/home/a_parida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/home/a_parida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


Epoch 2: Generator Loss: 0.691207, Discriminator Loss: 1.382382
Epoch 3: Generator Loss: 0.691286, Discriminator Loss: 1.382216
Epoch 4: Generator Loss: 0.691363, Discriminator Loss: 1.382052
Epoch 5: Generator Loss: 0.691441, Discriminator Loss: 1.381887
Epoch 6: Generator Loss: 0.691518, Discriminator Loss: 1.381722
Epoch 7: Generator Loss: 0.691595, Discriminator Loss: 1.381558
Epoch 8: Generator Loss: 0.691672, Discriminator Loss: 1.381395
Epoch 9: Generator Loss: 0.691749, Discriminator Loss: 1.381232
Epoch 10: Generator Loss: 0.691826, Discriminator Loss: 1.381069
Epoch 11: Generator Loss: 0.691902, Discriminator Loss: 1.380906
Epoch 12: Generator Loss: 0.691978, Discriminator Loss: 1.380742
Epoch 13: Generator Loss: 0.692055, Discriminator Loss: 1.380578
Epoch 14: Generator Loss: 0.692131, Discriminator Loss: 1.380414
Epoch 15: Generator Loss: 0.692208, Discriminator Loss: 1.380249
Epoch 16: Generator Loss: 0.692284, Discriminator Loss: 1.380084
Epoch 17: Generator Loss: 0.69236

In [12]:
###Counting the learnable parameters
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print("Total No. Learnable Parameters for the GAN:",total_parameters)


Total No. Learnable Parameters for the GAN: 7790


In [13]:
tf.trainable_variables()

[<tf.Variable 'g_Wdeconv1:0' shape=(3, 3, 8, 16) dtype=float32_ref>,
 <tf.Variable 'g_Bdeconv1:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'g_Wdeconv2:0' shape=(4, 4, 4, 8) dtype=float32_ref>,
 <tf.Variable 'g_Bdeconv2:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'g_Wdeconv3:0' shape=(6, 6, 2, 4) dtype=float32_ref>,
 <tf.Variable 'g_Bdeconv3:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'g_Wdeconv4:0' shape=(12, 12, 2, 2) dtype=float32_ref>,
 <tf.Variable 'g_Bdeconv4:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'd_Wconv1:0' shape=(12, 12, 2, 2) dtype=float32_ref>,
 <tf.Variable 'd_Bconv1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'd_Wconv2:0' shape=(6, 6, 2, 4) dtype=float32_ref>,
 <tf.Variable 'd_Bconv2:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'd_Wconv3:0' shape=(4, 4, 4, 8) dtype=float32_ref>,
 <tf.Variable 'd_Bconv3:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'd_Wconv4:0' shape=(3, 3, 8, 16) dtype=float32_ref>,
 <tf.Variable 'd_Bconv4:0' shape=(16,) dtype